In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
sns.set(style="white")
from statsmodels.formula.api import ols

from math import exp
from scipy.stats import norm
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 300)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [2]:
df = pd.read_csv("proj_data.csv")

In [3]:
df.columns

Index(['Unnamed: 0', 'STATE', 'YEAR_x', 'GRAD_PERCENT', 'TOTAL_REVENUE',
       'LOCAL_REVENUE', 'STATE_REVENUE', 'FEDERAL_REVENUE', 'LOCAL_PERCENT',
       'FED_PERCENT', 'STATE_PERCENT', 'TOTAL_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'INSTRUCTION_EXPENDITURE'],
      dtype='object')

In [5]:
df.isnull().sum()

Unnamed: 0                      0
STATE                           0
YEAR_x                          0
GRAD_PERCENT                    7
TOTAL_REVENUE                   0
LOCAL_REVENUE                   0
STATE_REVENUE                   0
FEDERAL_REVENUE                 0
LOCAL_PERCENT                   0
FED_PERCENT                     0
STATE_PERCENT                   0
TOTAL_EXPENDITURE               0
SUPPORT_SERVICES_EXPENDITURE    0
INSTRUCTION_EXPENDITURE         0
dtype: int64

In [8]:
df.head(100)
# df.iloc[200:220]

,Unnamed: 0,STATE,YEAR_x,GRAD_PERCENT,TOTAL_REVENUE,LOCAL_REVENUE,STATE_REVENUE,FEDERAL_REVENUE,LOCAL_PERCENT,FED_PERCENT,STATE_PERCENT,TOTAL_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,INSTRUCTION_EXPENDITURE
0,0,ALABAMA,2010,72.0,7281058.0,2346819.0,3800128.0,1134111.0,0.322318,0.155762,0.521920,7647571.0,2308399.0,3900492.0
1,1,ALASKA,2010,68.0,2230338.0,506695.0,1356673.0,366970.0,0.227183,0.164536,0.608281,2371251.0,856333.0,1151630.0
2,2,ARIZONA,2010,78.0,8828122.0,4108728.0,3139588.0,1579806.0,0.465414,0.178952,0.355635,8812967.0,2949514.0,4223152.0
3,3,ARKANSAS,2010,81.0,5069272.0,604422.0,3677112.0,787738.0,0.119233,0.155395,0.725373,5520387.0,1595730.0,2521528.0
4,4,CALIFORNIA,2010,76.0,67356719.0,23466828.0,34173911.0,9715980.0,0.348396,0.144247,0.507357,71227111.0,20704755.0,34657275.0
5,5,COLORADO,2010,74.0,8793676.0,4261571.0,3817875.0,714230.0,0.484618,0.081221,0.434161,8914338.0,2807231.0,4232076.0
6,6,CONNECTICUT,2010,83.0,9726922.0,5669948.0,3238401.0,818573.0,0.582913,0.084155,0.332932,9542108.0,2768520.0,5242817.0
7,7,DELAWARE,2010,78.0,1733458.0,560452.0,992422.0,180584.0,0.323314,0.104176,0.572510,1758988.0,499586.0,901200.0
8,8,DISTRICT_OF_COLUMBIA,2010,59.0,1195934.0,1115349.0,0.0,80585.0,0.932618,0.067382,0.000000,1290048.0,425379.0,526469.0
9,9,DISTRICT_OF_COLUMBIA,2010,71.0,1195934.0,1115349.0,0.0,80585.0,0.932618,0.067382,0.000000,1290048.0,425379.0,526469.0


In [9]:
df.shape

(255, 14)

In [12]:
cost_liv_dict = { 'Hawaii': 168.6,
                'District of Columbia': 146.5,
                'New York': 135.6,
                'California': 134.3,
                'Alaska':133.5,
                'Connecticut': 131.8,
                'Massachusetts': 130.4,
                'Oregon': 129.5,
                'New Jersey': 125.6,
                'Vermont': 123.8,
                'Rhode Island': 123.3,
                'Maryland': 121.1,
                'New Hampshire': 118.2,
                'Maine': 114.7,
                'Nevada': 106.5,
                'Washington': 106.0,
                'West Virginia': 103.7,
                'Pennsylvania': 103.0,
                'Delaware': 102.8,
                'Montana': 102.7,
                'South Dakota': 102.5,
                'Colorado': 101.9,
                'Minnesota': 101.5,
                'North Dakota': 101.2,
                'Florida': 100.5,
                'Arizona': 98.8,
                'Wisconsin': 98.1,
                'South Carolina': 97.5,
                'Illinois': 96.5,
                'North Carolina': 95.8,

                'Virginia': 94.5,
                'Louisiana': 93.4,
                'Wyoming': 92.8,
                'Texas': 92.6,
                'Ohio': 92.5,
                'Utah': 92.4,
                'Nebraska': 92.3,
                'Iowa': 92.0,
                'Georgia': 91.7,
                'Missouri': 91.5,

                'Arkansas': 91.4,
                'Michigan': 91.2,
                'Kansas': 90.9,
                'Tennessee': 90.3,
                'Alabama': 90.2,
                'Kentucky': 90.0,
                'Oklahoma': 89.7,
                'Idaho': 88.2,
                'Indiana': 88.0,
                'Mississippi': 83.5,

                'New Mexico': 100.0   
    
                }


In [14]:
cost_liv_dict = dict((k.upper().replace(' ','_'),v) for k, v in cost_liv_dict.items())
cost_liv_dict

{'HAWAII': 168.6,
 'DISTRICT_OF_COLUMBIA': 146.5,
 'NEW_YORK': 135.6,
 'CALIFORNIA': 134.3,
 'ALASKA': 133.5,
 'CONNECTICUT': 131.8,
 'MASSACHUSETTS': 130.4,
 'OREGON': 129.5,
 'NEW_JERSEY': 125.6,
 'VERMONT': 123.8,
 'RHODE_ISLAND': 123.3,
 'MARYLAND': 121.1,
 'NEW_HAMPSHIRE': 118.2,
 'MAINE': 114.7,
 'NEVADA': 106.5,
 'WASHINGTON': 106.0,
 'WEST_VIRGINIA': 103.7,
 'PENNSYLVANIA': 103.0,
 'DELAWARE': 102.8,
 'MONTANA': 102.7,
 'SOUTH_DAKOTA': 102.5,
 'COLORADO': 101.9,
 'MINNESOTA': 101.5,
 'NORTH_DAKOTA': 101.2,
 'FLORIDA': 100.5,
 'ARIZONA': 98.8,
 'WISCONSIN': 98.1,
 'SOUTH_CAROLINA': 97.5,
 'ILLINOIS': 96.5,
 'NORTH_CAROLINA': 95.8,
 'VIRGINIA': 94.5,
 'LOUISIANA': 93.4,
 'WYOMING': 92.8,
 'TEXAS': 92.6,
 'OHIO': 92.5,
 'UTAH': 92.4,
 'NEBRASKA': 92.3,
 'IOWA': 92.0,
 'GEORGIA': 91.7,
 'MISSOURI': 91.5,
 'ARKANSAS': 91.4,
 'MICHIGAN': 91.2,
 'KANSAS': 90.9,
 'TENNESSEE': 90.3,
 'ALABAMA': 90.2,
 'KENTUCKY': 90.0,
 'OKLAHOMA': 89.7,
 'IDAHO': 88.2,
 'INDIANA': 88.0,
 'MISSISSIPPI':

In [17]:
df['Cost_living'] = df['STATE'].map(cost_liv_dict)


In [18]:
df

,Unnamed: 0,STATE,YEAR_x,GRAD_PERCENT,TOTAL_REVENUE,LOCAL_REVENUE,STATE_REVENUE,FEDERAL_REVENUE,LOCAL_PERCENT,FED_PERCENT,STATE_PERCENT,TOTAL_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,INSTRUCTION_EXPENDITURE,Cost_living
0,0,ALABAMA,2010,72.0,7281058.0,2346819.0,3800128.0,1134111.0,0.322318,0.155762,0.521920,7647571.0,2308399.0,3900492.0,90.2
1,1,ALASKA,2010,68.0,2230338.0,506695.0,1356673.0,366970.0,0.227183,0.164536,0.608281,2371251.0,856333.0,1151630.0,133.5
2,2,ARIZONA,2010,78.0,8828122.0,4108728.0,3139588.0,1579806.0,0.465414,0.178952,0.355635,8812967.0,2949514.0,4223152.0,98.8
3,3,ARKANSAS,2010,81.0,5069272.0,604422.0,3677112.0,787738.0,0.119233,0.155395,0.725373,5520387.0,1595730.0,2521528.0,91.4
4,4,CALIFORNIA,2010,76.0,67356719.0,23466828.0,34173911.0,9715980.0,0.348396,0.144247,0.507357,71227111.0,20704755.0,34657275.0,134.3
5,5,COLORADO,2010,74.0,8793676.0,4261571.0,3817875.0,714230.0,0.484618,0.081221,0.434161,8914338.0,2807231.0,4232076.0,101.9
6,6,CONNECTICUT,2010,83.0,9726922.0,5669948.0,3238401.0,818573.0,0.582913,0.084155,0.332932,9542108.0,2768520.0,5242817.0,131.8
7,7,DELAWARE,2010,78.0,1733458.0,560452.0,992422.0,180584.0,0.323314,0.104176,0.572510,1758988.0,499586.0,901200.0,102.8
8,8,DISTRICT_OF_COLUMBIA,2010,59.0,1195934.0,1115349.0,0.0,80585.0,0.932618,0.067382,0.000000,1290048.0,425379.0,526469.0,146.5
9,9,DISTRICT_OF_COLUMBIA,2010,71.0,1195934.0,1115349.0,0.0,80585.0,0.932618,0.067382,0.000000,1290048.0,425379.0,526469.0,146.5


In [21]:
state_popul = {'HAWAII': 1.43,
     'DISTRICT_OF_COLUMBIA': 0.672,
     'NEW_YORK': 19.80,
     'CALIFORNIA': 39.1,
     'ALASKA': 0.738,
     'CONNECTICUT': 3.59,
     'MASSACHUSETTS': 6.79,
     'OREGON': 4.03,
     'NEW_JERSEY': 8.96,
     'VERMONT': 0.626,
     'RHODE_ISLAND': 1.06,
     'MARYLAND': 6.01,
     'NEW_HAMPSHIRE': 1.33,
     'MAINE': 1.33,
     'NEVADA': 2.89,
     'WASHINGTON': 7.17,
     'WEST_VIRGINIA': 1.84,
     'PENNSYLVANIA': 12.8,
     'DELAWARE': 0.945,
     'MONTANA': 1.03,
     'SOUTH_DAKOTA': 0.858,
     'COLORADO': 5.46,
     'MINNESOTA': 5.49,
     'NORTH_DAKOTA': 0.756,
     'FLORIDA': 20.27,
     'ARIZONA': 6.83,
     'WISCONSIN': 5.77,
     'SOUTH_CAROLINA': 4.90,
     'ILLINOIS': 12.86,
     'NORTH_CAROLINA': 10.0,
     'VIRGINIA': 8.38,
     'LOUISIANA': 4.67,
     'WYOMING': 0.586,
     'TEXAS': 27.47,
     'OHIO': 11.6,
     'UTAH': 3.00,
     'NEBRASKA': 1.90,
     'IOWA': 3.12,
     'GEORGIA': 10.2,
     'MISSOURI': 6.08,
     'ARKANSAS': 2.98,
     'MICHIGAN': 9.92,
     'KANSAS': 2.91,
     'TENNESSEE': 6.60,
     'ALABAMA': 4.86,
     'KENTUCKY': 4.42,
     'OKLAHOMA': 3.91,
     'IDAHO': 1.65,
     'INDIANA': 6.62,
     'MISSISSIPPI': 2.99,
     'NEW_MEXICO': 2.09}

In [22]:
state_popul

{'HAWAII': 1.43,
 'DISTRICT_OF_COLUMBIA': 0.672,
 'NEW_YORK': 19.8,
 'CALIFORNIA': 39.1,
 'ALASKA': 0.738,
 'CONNECTICUT': 3.59,
 'MASSACHUSETTS': 6.79,
 'OREGON': 4.03,
 'NEW_JERSEY': 8.96,
 'VERMONT': 0.626,
 'RHODE_ISLAND': 1.06,
 'MARYLAND': 6.01,
 'NEW_HAMPSHIRE': 1.33,
 'MAINE': 1.33,
 'NEVADA': 2.89,
 'WASHINGTON': 7.17,
 'WEST_VIRGINIA': 1.84,
 'PENNSYLVANIA': 12.8,
 'DELAWARE': 0.945,
 'MONTANA': 1.03,
 'SOUTH_DAKOTA': 0.858,
 'COLORADO': 5.46,
 'MINNESOTA': 5.49,
 'NORTH_DAKOTA': 0.756,
 'FLORIDA': 20.27,
 'ARIZONA': 6.83,
 'WISCONSIN': 5.77,
 'SOUTH_CAROLINA': 4.9,
 'ILLINOIS': 12.86,
 'NORTH_CAROLINA': 10.0,
 'VIRGINIA': 8.38,
 'LOUISIANA': 4.67,
 'WYOMING': 0.586,
 'TEXAS': 27.47,
 'OHIO': 11.6,
 'UTAH': 3.0,
 'NEBRASKA': 1.9,
 'IOWA': 3.12,
 'GEORGIA': 10.2,
 'MISSOURI': 6.08,
 'ARKANSAS': 2.98,
 'MICHIGAN': 9.92,
 'KANSAS': 2.91,
 'TENNESSEE': 6.6,
 'ALABAMA': 4.86,
 'KENTUCKY': 4.42,
 'OKLAHOMA': 3.91,
 'IDAHO': 1.65,
 'INDIANA': 6.62,
 'MISSISSIPPI': 2.99,
 'NEW_MEXICO

In [23]:
df['State_popul'] = df['STATE'].map(state_popul)


In [24]:
df

,Unnamed: 0,STATE,YEAR_x,GRAD_PERCENT,TOTAL_REVENUE,LOCAL_REVENUE,STATE_REVENUE,FEDERAL_REVENUE,LOCAL_PERCENT,FED_PERCENT,STATE_PERCENT,TOTAL_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,INSTRUCTION_EXPENDITURE,Cost_living,State_popul
0,0,ALABAMA,2010,72.0,7281058.0,2346819.0,3800128.0,1134111.0,0.322318,0.155762,0.521920,7647571.0,2308399.0,3900492.0,90.2,4.860
1,1,ALASKA,2010,68.0,2230338.0,506695.0,1356673.0,366970.0,0.227183,0.164536,0.608281,2371251.0,856333.0,1151630.0,133.5,0.738
2,2,ARIZONA,2010,78.0,8828122.0,4108728.0,3139588.0,1579806.0,0.465414,0.178952,0.355635,8812967.0,2949514.0,4223152.0,98.8,6.830
3,3,ARKANSAS,2010,81.0,5069272.0,604422.0,3677112.0,787738.0,0.119233,0.155395,0.725373,5520387.0,1595730.0,2521528.0,91.4,2.980
4,4,CALIFORNIA,2010,76.0,67356719.0,23466828.0,34173911.0,9715980.0,0.348396,0.144247,0.507357,71227111.0,20704755.0,34657275.0,134.3,39.100
5,5,COLORADO,2010,74.0,8793676.0,4261571.0,3817875.0,714230.0,0.484618,0.081221,0.434161,8914338.0,2807231.0,4232076.0,101.9,5.460
6,6,CONNECTICUT,2010,83.0,9726922.0,5669948.0,3238401.0,818573.0,0.582913,0.084155,0.332932,9542108.0,2768520.0,5242817.0,131.8,3.590
7,7,DELAWARE,2010,78.0,1733458.0,560452.0,992422.0,180584.0,0.323314,0.104176,0.572510,1758988.0,499586.0,901200.0,102.8,0.945
8,8,DISTRICT_OF_COLUMBIA,2010,59.0,1195934.0,1115349.0,0.0,80585.0,0.932618,0.067382,0.000000,1290048.0,425379.0,526469.0,146.5,0.672
9,9,DISTRICT_OF_COLUMBIA,2010,71.0,1195934.0,1115349.0,0.0,80585.0,0.932618,0.067382,0.000000,1290048.0,425379.0,526469.0,146.5,0.672
